In [ ]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os


from operator import itemgetter

import numpy as np
import scipy

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import skimage.io

from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [ ]:
# Build model and train

# Split data into train and validation

# JØRGEN KAN BEGYNNE BYGGE DETTE

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from dask.diagnostics import ProgressBar

bern = BernoulliNB()
with ProgressBar():
    bern.fit(x_train_df, y_train_df)